# Lab. 3: Momentum I

In this lab we will explore the signal research proccess with the classic Jegadeesh and Titman momentum signal.

We will explore:
- How to compute signals with point in time compliance in mind.
- The need to filter out low priced securities.
- How to create decile and spread portfolios.
- Strategy performance metrics.

## Imports

In [2]:
import sf_quant.data as sfd
import polars as pl
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data

We will be looking at our investment universe from 1996 to 2024.

This should take around 10 seconds to run.

In [3]:
start = dt.date(1996, 1, 1)
end = dt.date(2024, 12, 31)

columns = [
    'date',
    'barrid',
    'ticker',
    'price',
    'return',
    'market_cap'
]

data = sfd.load_assets(
    start=start,
    end=end,
    in_universe=True,
    columns=columns
)

data

date,barrid,ticker,price,return,market_cap
date,str,str,f64,f64,f64
2013-07-31,"""USA06Z1""","""MDXG""",6.26,-0.1595,6.006157e8
2013-08-01,"""USA06Z1""","""MDXG""",6.32,0.9585,6.0865392e8
2013-08-02,"""USA06Z1""","""MDXG""",6.31,-0.1582,6.0769086e8
2013-08-05,"""USA06Z1""","""MDXG""",6.45,2.2187,6.211737e8
2013-08-06,"""USA06Z1""","""MDXG""",6.29,-2.4806,6.0576474e8
…,…,…,…,…,…
2024-12-24,"""USBQOR1""","""ECG""",70.58,2.5872,3.5976e9
2024-12-26,"""USBQOR1""","""ECG""",73.61,4.293,3.7521e9
2024-12-27,"""USBQOR1""","""ECG""",69.85,-5.108,3.5604e9


## Compute the Momentum Signal

## Instructions

- Compute momentum for each security and date as the rolling 230 day return (you can just use log returns here).
- Shift the momentum signal 22 days. This will results in the 11 month return from t-12 to t-2.

In [4]:
def task_compute_momentum(data: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the t_12 to t_2 momentum signal for each secrutiy and date combination.
    
    Args:
        data (pl.DataFrame): Data frame containing date, barrid, price, and return columns.
    
    Returns:
        pl.DataFrame: Data frame with columns date, barrid, price, return, and momentum columns.
    """
    return (
        data
        .with_columns(
            pl.col('return').truediv(100)
        )
        .with_columns(
            log_return = pl.col('return').log1p()
        )
        .sort('barrid', 'date')
        .with_columns(
            momentum = pl.col('log_return').rolling_sum(window_size=230).over('barrid')
        )
        .with_columns(
            pl.col('momentum').shift(22).over('barrid')
        )
    )

momentum = task_compute_momentum(data)

momentum

date,barrid,ticker,price,return,market_cap,log_return,momentum
date,str,str,f64,f64,f64,f64,f64
2013-07-31,"""USA06Z1""","""MDXG""",6.26,-0.001595,6.006157e8,-0.001596,null
2013-08-01,"""USA06Z1""","""MDXG""",6.32,0.009585,6.0865392e8,0.009539,null
2013-08-02,"""USA06Z1""","""MDXG""",6.31,-0.001582,6.0769086e8,-0.001583,null
2013-08-05,"""USA06Z1""","""MDXG""",6.45,0.022187,6.211737e8,0.021944,null
2013-08-06,"""USA06Z1""","""MDXG""",6.29,-0.024806,6.0576474e8,-0.025119,null
…,…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""","""ECG""",70.58,0.025872,3.5976e9,0.025543,null
2024-12-26,"""USBQOR1""","""ECG""",73.61,0.04293,3.7521e9,0.042034,null
2024-12-27,"""USBQOR1""","""ECG""",69.85,-0.05108,3.5604e9,-0.052431,null


In [6]:
first_values = (
    momentum
    .filter(pl.col('momentum').is_not_null())
    .group_by('barrid')
    .head(1)
)

print(first_values)

shape: (9_383, 8)
┌─────────┬────────────┬────────┬────────┬───────────┬─────────────┬────────────┬───────────┐
│ barrid  ┆ date       ┆ ticker ┆ price  ┆ return    ┆ market_cap  ┆ log_return ┆ momentum  │
│ ---     ┆ ---        ┆ ---    ┆ ---    ┆ ---       ┆ ---         ┆ ---        ┆ ---       │
│ str     ┆ date       ┆ str    ┆ f64    ┆ f64       ┆ f64         ┆ f64        ┆ f64       │
╞═════════╪════════════╪════════╪════════╪═══════════╪═════════════╪════════════╪═══════════╡
│ USA06Z1 ┆ 2014-07-30 ┆ MDXG   ┆ 7.1    ┆ 0.009957  ┆ 7.410696e8  ┆ 0.009908   ┆ 0.101525  │
│ USA0771 ┆ 2015-06-29 ┆ UIHC   ┆ 15.82  ┆ -0.004405 ┆ 3.3971e8    ┆ -0.004415  ┆ -0.126918 │
│ USA0BJ1 ┆ 2013-07-02 ┆ BLMT   ┆ 13.22  ┆ -0.011958 ┆ 1.2126706e8 ┆ -0.01203   ┆ 0.086478  │
│ USA0C01 ┆ 2017-06-29 ┆ WFBI   ┆ 34.54  ┆ 0.009646  ┆ 4.2155e8    ┆ 0.0096     ┆ 0.506896  │
│ USA0C11 ┆ 2012-10-31 ┆ FBHS   ┆ 28.44  ┆ 0.026344  ┆ 4.5398e9    ┆ 0.026003   ┆ 0.611123  │
│ …       ┆ …          ┆ …      ┆ …      ┆

## Price and Momentum Filter

When doing backtesting strategies it's helpful to drop securities with a price below $5.

- Create a variable price_lag that is the previous days price for each security.
- Filter securities to ones with a lagged price greater than 5.
- Drop null momentum values since we won't trade securities where we don't have a signal.

In [7]:
def task_price_filter(momentum: pl.DataFrame) -> pl.DataFrame:
    """
    Filter the universe to lagged price greater than 5 and non-null momentum.
    
    Args:
        momentum (pl.DataFrame): Data frame with columns date, barrid, price, return, and momentum columns.
    
    Returns:
        pl.DataFrame: Data frame with columns date, barrid, price, return, and momentum columns.
    """
    return (
        momentum
        .sort('barrid', 'date')
        .with_columns(
            price_lag = pl.col('price').shift(1).over('barrid')
        )
        .filter(
            pl.col('price_lag').gt(5),
            pl.col('momentum').is_not_null()
        )
        .sort('barrid', 'date')
    )

price_filter = task_price_filter(momentum)

price_filter

date,barrid,ticker,price,return,market_cap,log_return,momentum,price_lag
date,str,str,f64,f64,f64,f64,f64,f64
2014-07-30,"""USA06Z1""","""MDXG""",7.1,0.009957,7.410696e8,0.009908,0.101525,7.03
2014-07-31,"""USA06Z1""","""MDXG""",6.91,-0.026761,7.2123816e8,-0.027126,0.124505,7.1
2014-08-01,"""USA06Z1""","""MDXG""",6.81,-0.014472,7.1080056e8,-0.014578,0.167176,6.91
2014-08-04,"""USA06Z1""","""MDXG""",7.08,0.039648,7.3898208e8,0.038882,0.159344,6.81
2014-08-05,"""USA06Z1""","""MDXG""",7.05,-0.004237,7.358508e8,-0.004246,0.14279,7.08
…,…,…,…,…,…,…,…,…
2024-12-27,"""USBPJV1""","""NLOP""",30.81,-0.019414,4.5553e8,-0.019605,0.527437,31.42
2024-12-30,"""USBPJV1""","""NLOP""",31.08,0.008763,4.5952e8,0.008725,0.522813,30.81
2024-12-31,"""USBPJV1""","""NLOP""",31.21,0.004183,4.6144e8,0.004174,0.552793,31.08


## Create Decile Portfolios

### Instructions

- For each date divide all securities into 10 deciles based on their momentum score. The highest momentum securities should be in bin 9 and the lowest momentum securities should be in bin 0. The `.qcut()` expression will be helpful here.


In [ ]:
def task_momentum_bins(price_filter: pl.DataFrame) -> pl.DataFrame:
    """
    Bin the securities into deciles for each date by momentum.
    
    Args:
        price_filter (pl.DataFrame): Data frame with columns date, barrid, price, return, and momentum columns.
    
    Returns:
        pl.DataFrame: Data frame with columns date, barrid, price, return, momentum and bin columns.
    """
    # TODO: Finish this function
    pass

momentum_bins = task_momentum_bins(price_filter)

momentum_bins

## Equal Weight Portfolios

For each date and bin combo compute the equal weight return.

### Instructions
- Use `.group_by()` to find the average return for each decile on each date.
- Pivot the data frame into index=date, columns=bins, and values=returns (`.pivot()`)
- Compute the spread portfolio as the return of bin 9 minus the return of bin 0

In [ ]:
def task_equal_weight_portfolios(momentum_bins: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the equal weight return for each bin on each date.
    
    Args:
        momentum_bins (pl.DataFrame): Data frame with columns date, barrid, price, return, momentum, and bin columns.
    
    Returns:
        pl.DataFrame: Data frame with date as the index, bins as the columns, and returns as the values.
    """
    # TODO: Finish this function
    pass

momentum_portfolios = task_equal_weight_portfolios(momentum_bins)

momentum_portfolios

## Portfolio Returns

Compute the cumulative returns of each bin.

### Instructions
- Use the `.unpivot()` method to put our data frame back into long format.
- Compute the cumulative log return.
- Put the return columns into percent space by multiplying by 100.

In [ ]:
def task_cumulative_returns(momentum_portfolios: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the equal weight return for each bin on each date.
    
    Args:
        momentum_portfolios (pl.DataFrame): Data frame with columns date, barrid, price, return, momentum, and bin columns.
    
    Returns:
        pl.DataFrame: Data frame with date, bin, return, and cumulative_log_return columns.
    """
    # TODO: Finish this function
    pass

momentum_returns = task_cumulative_returns(momentum_portfolios)

momentum_returns

## Performance Analysis

### Instructions
- Chart the cumulative log returns of the 10 decile portfolios and the spread portfolio.
- Create a table with average daily return (annualized), volatility (annualized), and sharpe ratio (annualized) for each portfolio.

In [ ]:
# TODO: Chart the cumulative log returns of each portfolio.

In [ ]:
# TODO: Create a table of summary metrics